# Running MCFE on target circuits

In [1]:
import pygsti
from collections import defaultdict
import numpy as np
import time

In [2]:
# Create pyGSTi circs
unmapped_circs = [pygsti.circuits.Circuit([["Gxpi2", "Q0"], ["Gypi2", "Q1"]]),pygsti.circuits.Circuit([["Gypi2", "Q0"], ["Gxpi2", "Q1"]])]

### Map circuits to device connectivity and U3-CX gate set

This step will be different depending on what architecture you are using. For this example, we are using an IBM device. You need to end up with pyGSTi circuits in a U3-CX gate set so that circuit mirroring can be performed.

In [3]:
mapped_circs = defaultdict(list)

import qiskit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager as _pass_manager
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke, FakeAthensV2

fake_backend = FakeAthensV2()

pm = _pass_manager(coupling_map=fake_backend.coupling_map, basis_gates=['u3', 'cx'], optimization_level=0)


for i, circ in enumerate(unmapped_circs):
    # Convert from pyGSTi to Qiskit
    # Comment these lines out and do qiskit_circ = circ if passing in Qiskit
    pygsti_openqasm_circ = circ.convert_to_openqasm(block_between_layers=True, include_delay_on_idle=False)
    # print(pygsti_openqasm_circ)
    qiskit_circ = qiskit.QuantumCircuit.from_qasm_str(pygsti_openqasm_circ)

    # print(qiskit_circ.draw())

    mapped_qiskit_circ = pm.run(qiskit_circ)

    # print(mapped_qiskit_circ.draw())
    pygsti_circ = pygsti.circuits.Circuit.from_qiskit(mapped_qiskit_circ)
    # print(pygsti_circ)

    mapped_circ = pygsti_circ

    metadata = {'width': len(mapped_circ.line_labels), 'physical_depth': mapped_circ.depth, 'dropped_gates': 0, 'id': i}
    mapped_circs[mapped_circ] += [metadata]


unmirrored_design = pygsti.protocols.FreeformDesign(mapped_circs)


/home/ndsieki/pygsti_development/pyGSTi/pygsti/circuits/circuit.py:4086: UserWarning: skipping barrier
  _warnings.warn('skipping barrier')
/home/ndsieki/pygsti_development/pyGSTi/pygsti/circuits/circuit.py:4082: UserWarning: skipping measure
  _warnings.warn('skipping measure')


### Mirror circuit generation

We use Pauli random compiling (`pauli_rc`) here. Central Pauli (`central_pauli`) is also an option.

In [4]:
# Highly recommended to seed all RNG
mcfe_rand_state = np.random.RandomState(20240718)

start = time.time()
mirror_design = pygsti.protocols.mirror_edesign.make_mirror_edesign(
    unmirrored_design,
    num_mcs_per_circ=100,
    num_ref_per_qubit_subset=100,
    mirroring_strategy='pauli_rc',
    rand_state=mcfe_rand_state)
print(f'Mirroring time:', time.time() - start)

Sampling mirror circuits:   0%|          | 0/2 [00:00<?, ?it/s]

using provided edesign for both reference and test compilations


Sampling mirror circuits:  50%|#####     | 1/2 [00:00<00:00,  4.13it/s]

using provided edesign for both reference and test compilations


Sampling mirror circuits: 100%|##########| 2/2 [00:00<00:00,  3.38it/s]


Sampling reference circuits for width 5 with 1 subsets


Sampling reference circuits for subset ('Q0', 'Q1', 'Q2', 'Q3', 'Q4'): 100%|##########| 100/100 [00:00<00:00, 2232.96it/s]

Mirroring time: 0.6630532741546631


We have created the MCFE experiment design.

### Run the Edesign

This example will run the edesign on a fake IBM backend, but this is not strictly required. This step needs to generate a `ProtocolData(edesign=mirror_edesign, dataset=circuit_counts_data)` where `mirror_edesign` is the variable defined earlier and `circuit_counts_data` is a `DataSet` that contains the outcomes for each circuit.

In [5]:
from pygsti.extras.devices import ExperimentalDevice
from pygsti.extras import devices, ibmq

device = ExperimentalDevice.from_qiskit_backend(fake_backend)
# device = ExperimentalDevice.from_legacy_device('ibm_brisbane')
pspec = device.create_processor_spec(['Gc{}'.format(i) for i in range(24)] + ['Gcnot'])

start = time.time()
#exp = ibmq.IBMQExperiment(combined_mirror_design, pspec, circuits_per_batch=300, num_shots=1024, seed=20240718, disable_checkpointing=True)
exp = ibmq.IBMQExperiment(mirror_design, pspec, circuits_per_batch=300, num_shots=1024, seed=20240718, checkpoint_override=True)
print(time.time() - start)

0.14238238334655762


In [6]:
from qiskit_aer import AerSimulator

sim_backend = AerSimulator.from_backend(fake_backend)

qiskit_pass_kwargs = {
    'optimization_level': 0,
    'routing_method': 'none'
}

qasm_convert_kwargs = {
    'num_qubits': pspec.num_qubits,
    'standard_gates_version': 'x-sx-rz',
    'include_delay_on_idle': True
}

start = time.time()
exp.transpile(sim_backend, qiskit_pass_kwargs, qasm_convert_kwargs)
end = time.time()
print(f'Total transpilation time: {end - start}')

100%|██████████| 2/2 [00:17<00:00,  8.62s/it]

Total transpilation time: 17.268017530441284


In [7]:
exp.submit(sim_backend)

Submitting batch 1
  - Job ID is 0d0c4c76-ba08-4a5e-8250-63c37651c54d
  - Failed to get queue position for batch 1
Submitting batch 2
  - Job ID is bc708a2c-0e8d-4359-a229-a4b5f2a871c6
  - Failed to get queue position for batch 2


In [8]:
start = time.time()
exp.batch_results = []
exp.retrieve_results()
end = time.time()
print(end - start)

Querying IBMQ for results objects for batch 1...
Querying IBMQ for results objects for batch 2...
16.18717908859253


In [9]:
data = exp.data

### Compute process fidelity for each circuit

The cell below contains functions that ultimately compute a dataframe containing the process fidelities.

In [10]:
import pandas as pd
import tqdm

def hamming_distance_counts(dsrow, circ, idealout):
    nQ = len(circ.line_labels)  # number of qubits
    assert(nQ == len(idealout[-1]))
    hamming_distance_counts = np.zeros(nQ + 1, float)
    if dsrow.total > 0:
        for outcome_lbl, counts in dsrow.counts.items():
            outbitstring = outcome_lbl[-1]
            hamming_distance_counts[pygsti.tools.rbtools.hamming_distance(outbitstring, idealout[-1])] += counts
    return hamming_distance_counts

def adjusted_success_probability(hamming_distance_counts):
    if np.sum(hamming_distance_counts) == 0.: 
        return 0.
    else:
        hamming_distance_pdf = np.array(hamming_distance_counts) / np.sum(hamming_distance_counts)
        adjSP = np.sum([(-1 / 2)**n * hamming_distance_pdf[n] for n in range(len(hamming_distance_pdf))])
        return adjSP

def effective_polarization(hamming_distance_counts):
    n = len(hamming_distance_counts) - 1 
    asp = adjusted_success_probability(hamming_distance_counts)
    
    return (4**n * asp - 1)/(4**n - 1)

def polarization_to_fidelity(p, n): 
    return 1 - (4**n - 1)*(1 - p)/4**n

def fidelity_to_polarization(f, n):
    return 1 - (4**n)*(1 - f)/(4**n - 1)

def predicted_process_fidelity(bare_rc_effective_pols, rc_rc_effective_pols, reference_effective_pols, n):

    a = np.mean(bare_rc_effective_pols)
    b = np.mean(rc_rc_effective_pols)
    c = np.mean(reference_effective_pols)
    if c <= 0.:
        return np.nan  # raise ValueError("Reference effective polarization zero or smaller! Cannot estimate the process fidelity")
    elif b <= 0:
        return 0.
    else:
        return polarization_to_fidelity(a / np.sqrt(b * c), n)

def predicted_process_fidelity_for_central_pauli_mcs(central_pauli_effective_pols, reference_effective_pols, n):
    a = np.mean(central_pauli_effective_pols)
    c = np.mean(reference_effective_pols)
    if c <= 0.:
        return np.nan  # raise ValueError("Reference effective polarization zero or smaller! Cannot estimate the process fidelity")
    elif a <= 0:
        return 0.
    else:
        return polarization_to_fidelity(np.sqrt(a / c), n)
    
def calculate_vb_df(unmirrored_design, mirrored_data, verbose=True):
    # Run through and calculate all effective polarizations
    eff_pols = {k: {} for k in mirrored_data.edesign.keys()}
    
    # Stats about the base circuit
    u3_densities = {}
    cnot_densities = {}
    cnot_counts = {}
    cnot_depths = {}
    pygsti_depths = {}
    idling_qubits = {}
    dropped_gates = {}
    occurrences = {}

    # Get a dict going from id to circuit for easy lookup
    reverse_circ_ids = {}
    for k,v in unmirrored_design.aux_info.items():
        if isinstance(v, dict):
            reverse_circ_ids[v['id']] = k
        else:
            for entry in v:
                reverse_circ_ids[entry['id']] = k
    seen_keys = set()

    num_circs = len(mirrored_data.dataset)
    for c in tqdm.tqdm(mirrored_data.dataset, ascii=True, desc='Calculating effective polarizations'):
        for edkey, ed in mirrored_data.edesign.items():
            auxlist = ed.aux_info.get(c, None)
            if auxlist is None:
                continue
            elif isinstance(auxlist, dict):
                auxlist = [auxlist]

            for aux in auxlist:
                if edkey.endswith('ref'):
                    # For reference circuits, only width matters, so aggregate on that now
                    key = aux['width']
                else:
                    key = (aux['base_aux']['width'], aux['base_aux']['physical_depth'], aux['base_aux']['id'])

                
                # Calculate effective polarization
                hdc = hamming_distance_counts(mirrored_data.dataset[c], c, (aux['idealout'],))
                ep = effective_polarization(hdc)
                
                # Append to other mirror circuit samples
                eps = eff_pols[edkey].get(key, [])
                eps.append(ep)
                eff_pols[edkey][key] = eps

                if edkey.endswith('ref') or key in seen_keys:
                    # Skip statistic gathering for reference circuits or base circuits we've seen already
                    continue

                orig_circ = reverse_circ_ids[key[2]]

                u3_count = 0
                cnot_count = 0
                cnot_depth = 0
                for i in range(orig_circ.depth):
                    layer_cnot_depth = 0
                    for gate in orig_circ._layer_components(i):
                        if gate.name == 'Gu3':
                            u3_count += 1
                        elif gate.name == 'Gcnot':
                            cnot_count += 2
                            layer_cnot_depth = 1
                    cnot_depth += layer_cnot_depth
                
                if cnot_count == 0:
                    cnot_count = 0.01
                    
                u3_densities[key] = u3_count / (key[0]*key[1])
                cnot_densities[key] = cnot_count / (key[0]*key[1])
                cnot_counts[key] = cnot_count / 2 # Want 1 for each CNOT
                cnot_depths[key] = cnot_depth
                pygsti_depths[key] = orig_circ.depth
                idling_qubits[key] = len(orig_circ.idling_lines())
                dropped_gates[key] = aux['base_aux']['dropped_gates']
                occurrences[key] = len(auxlist)

                seen_keys.add(key)

    # Calculate process fidelities
    df_data = {}
    for i, key in enumerate(sorted(list(seen_keys), key=lambda x: x[2])):
        cp_pfid = cp_pol = cp_success_prob = None
        if 'cp' in mirrored_data.edesign and 'cpref' in mirrored_data.edesign:
            if verbose and i == 0: print('Central pauli data detected, computing CP process fidelity')
            cp_pfid = predicted_process_fidelity_for_central_pauli_mcs(eff_pols['cp'][key], eff_pols['cpref'][key[0]], key[0])
            cp_pol = polarization_to_fidelity(cp_pfid, key[0])
            cp_success_prob = pygsti.protocols.vbdataframe.polarization_to_success_probability(cp_pol, key[0])
            
        rc_pfid = rc_pol = rc_success_prob = None
        if 'rr' in mirrored_data.edesign and 'br' in mirrored_data.edesign and 'ref' in mirrored_data.edesign:
            if verbose and i == 0: print('Random compilation data detected, computing RC process fidelity')
            rc_pfid = predicted_process_fidelity(eff_pols['br'][key], eff_pols['rr'][key], eff_pols['ref'][key[0]], key[0])
            rc_pol = polarization_to_fidelity(rc_pfid, key[0])
            rc_success_prob = pygsti.protocols.vbdataframe.polarization_to_success_probability(rc_pol, key[0])

        # Depth is doubled for conventions (same happens for RMC/PMC)
        df_data[i] = {'Width': key[0], 'Physical Depth': key[1], 'Circuit Id': key[2],
                      'U3 Density': u3_densities[key], 'CNOT Density': cnot_densities[key],
                      'CNOT Counts': cnot_counts[key], 'CNOT Depth': cnot_depths[key],
                      'U3+CNOT Depth': pygsti_depths[key], 'Dropped Gates': dropped_gates[key],
                      'Effective Width': key[0] - idling_qubits[key],
                      'CP Process Fidelity': cp_pfid, 'CP Polarization': cp_pol,
                      'CP Success Probability': cp_success_prob,
                      'RC Process Fidelity': rc_pfid, 'RC Polarization': rc_pol,
                      'RC Success Probability': rc_success_prob,
                      'Occurrences': occurrences[key], 'Total Counts': 1024}
    
    
    df = pd.DataFrame.from_dict(df_data, orient='index')
    df = df.sort_values(by='Circuit Id')
    df = df.reset_index(drop=True)

    return df

In [11]:
df = calculate_vb_df(unmirrored_design, data)

Calculating effective polarizations: 100%|##########| 500/500 [00:00<00:00, 2724.94it/s]

Random compilation data detected, computing RC process fidelity


If you used Central Pauli instead, you can swap `'RC Process Fidelity'` for `'CP Process Fidelity'` in the cell below.

In [12]:
process_fidelities = df['RC Process Fidelity']

In [13]:
process_fidelities.tolist()

[0.9950162628750925, 0.9961010286172316]